<a href="https://colab.research.google.com/github/MuhammadAbdullah95/My_Langgraph_Projects/blob/main/Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -qU langchain-pinecone langchain-google-genai langchain
!pip install google-auth

In [11]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
        api_key=PINECONE_API_KEY
    )


In [13]:
import time

index_name = "rag-project2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [14]:
from google.colab import userdata
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.auth import default

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)

In [15]:
vector = embeddings.embed_query("hello, world")


In [16]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# !pip install -qU langchain-pinecone langchain-google-genai langchain
# !pip install google-auth
# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
# import os
# from pinecone import Pinecone, ServerlessSpec

# pc = Pinecone(
#         api_key=PINECONE_API_KEY
#     )
# import time

# index_name = "rag-project2"  # change if desired

# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=768,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# index = pc.Index(index_name)
# from google.colab import userdata
# import os
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from google.auth import default

# # Get credentials from userdata or default service account
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# # Explicitly use API key for authentication
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=GEMINI_API_KEY # Use API key directly
# )

# # Remove the try-except block, as we are now using the API key directly

# vector = embeddings.embed_query("this is a rag text")

In [19]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

document_ids = vector_store.add_documents(documents=documents, ids=uuids)

In [20]:
document_ids

['c544c746-75a3-4b03-8384-ad159ae21480',
 '763b159d-7201-451d-8a11-25e27a282ed3',
 '696d2bd5-c12f-4ce9-903d-f50860b22de7',
 'f9fd8fb5-9b82-4c9f-b211-213710cca252',
 '21362800-13de-43dd-83bb-b25760920413',
 '7bca2ea8-0f06-4d33-91b4-f01e0c40a9ef',
 '715332f5-fc06-4788-9836-a9ecacdbdee4',
 'b76638e3-6b37-49f1-9114-735392fc1cf0',
 '561f0ea3-c7ac-4380-a59d-5da0ccdf34a0',
 '1f7184e0-782b-4a25-8add-a5744772be34']

In [22]:
vector_store.delete(ids=[uuids[-1]])

In [23]:
document_ids

['c544c746-75a3-4b03-8384-ad159ae21480',
 '763b159d-7201-451d-8a11-25e27a282ed3',
 '696d2bd5-c12f-4ce9-903d-f50860b22de7',
 'f9fd8fb5-9b82-4c9f-b211-213710cca252',
 '21362800-13de-43dd-83bb-b25760920413',
 '7bca2ea8-0f06-4d33-91b4-f01e0c40a9ef',
 '715332f5-fc06-4788-9836-a9ecacdbdee4',
 'b76638e3-6b37-49f1-9114-735392fc1cf0',
 '561f0ea3-c7ac-4380-a59d-5da0ccdf34a0',
 '1f7184e0-782b-4a25-8add-a5744772be34']

In [26]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [28]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
    filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.534239] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.534239] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [ ]:
def answer_to_user(query: str):

  # Vector Search
  result = vector_store.similarity_search(query, k=2)

  # Pass to model Vector Results + User Query

  final_answer = model(result + query)
  return final_answer

